<a href="https://colab.research.google.com/github/roberthouston14/Final_Praxis_Code/blob/main/GPT2_Data_Synthesizer_and_Balancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install pandas
!pip install nltk
!pip install textaugment
!pip install --upgrade accelerate
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.7 MB/s eta 0:00:00


In [ ]:
!pip uninstall -y transformers datasets torch pandas nltk textaugment accelerate



Found existing installation: transformers 4.32.1
Uninstalling transformers-4.32.1:
  Successfully uninstalled transformers-4.32.1
Found existing installation: pandas 2.1.0
Uninstalling pandas-2.1.0:
  Successfully uninstalled pandas-2.1.0
Found existing installation: nltk 3.8.1
Uninstalling nltk-3.8.1:
  Successfully uninstalled nltk-3.8.1
Found existing installation: textaugment 1.3.4
Uninstalling textaugment-1.3.4:
  Successfully uninstalled textaugment-1.3.4


In [2]:
# Transformers and datasets (from huggingface)
!pip install transformers  # If you wish to update, you can use --upgrade
!pip install datasets  # If you wish to update, you can use --upgrade

# Torch (PyTorch is usually pre-installed in Google Colab, omitting --upgrade)
# !pip install torch

# Pandas (already pre-installed in Google Colab, omitting --upgrade)
# !pip install pandas

# NLTK
!pip install nltk  # If you wish to update, you can use --upgrade

# TextAugment
!pip install textaugment  # If you wish to update, you can use --upgrade

# Accelerate
!pip install accelerate  # If you wish to update, you can use --upgrade

# Transformers with torch
!pip install transformers[torch]  # If you wish to update, you can use --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.9 MB/s eta 0:00

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Sep  4 00:15:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    42W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
import pandas as pd
import random
import nltk
import os
import nltk
import numpy as np
import warnings
from transformers import logging
logging.set_verbosity_warning()
warnings.filterwarnings("ignore", category=UserWarning, message="Input length of input_ids is .* but `max_length` is set to .*")
from nltk.translate.bleu_score import sentence_bleu
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers.optimization import AdamW
from datasets import load_dataset
from textaugment import Wordnet

nltk.download('punkt')

# Prompt user to input required information
def prompt_input():
    while True:
        try:
            input_file = input("Enter the path to the input file: ").strip()
            if not os.path.exists(input_file):
                print("The file does not exist. Please try again.")
                continue
            label_col = input("Enter the column name for the labels: ").strip()
            text_col = input("Enter the column name for the text: ").strip()
            return input_file, label_col, text_col
        except Exception as e:
            print(f"An error occurred: {e}. Please try again.")

# Calculate the imbalance of classes and determine the required samples for minority classes
def balance_classes(df, label_col):
    class_counts = df[label_col].value_counts()
    max_count = class_counts.max()

    minority_classes = class_counts[class_counts < max_count].index.tolist()
    required_samples = {cls: max_count - count for cls, count in zip(class_counts.index, class_counts) if cls in minority_classes}

    return minority_classes, required_samples

# Let the user select the GPT model to use for synthetic text generation
def select_model():
    model_choices = {
        1: "gpt2",
        2: "gpt2-medium",
        3: "gpt2-large",
        4: "gpt2-xl",
    }

    print("Select the GPT model to use:")
    for key, value in model_choices.items():
        print(f"{key}: {value}")

    while True:
        try:
            selected = int(input("Enter the number corresponding to the model you want to use: "))
            return model_choices[selected]
        except (ValueError, KeyError):
            print("Invalid input. Please enter a valid model number.")

# Fine-tune the GPT model with the provided seed texts
def train_gpt2_model(train_texts, model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    train_dataset = load_dataset("text", data_files=train_texts)["train"]
    train_dataset = train_dataset.map(lambda x: tokenizer(x['text'], truncation=True, padding=True), batched=True)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    model = AutoModelForCausalLM.from_pretrained(model_name)

    training_args = TrainingArguments(
        output_dir="./gpt2_finetuned",
        overwrite_output_dir=True,
        num_train_epochs=3,
        per_device_train_batch_size=8,
        save_steps=10_000,
        save_total_limit=2,
        learning_rate=5e-5,
        logging_steps=100,
        logging_dir="./logs",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )

    trainer.train()

    return tokenizer, model

def get_mean_std(tokenizer, texts):
    token_lengths = [len(tokenizer.encode(text)) for text in texts]
    mean = sum(token_lengths) / len(token_lengths)
    std = (sum([(length - mean) ** 2 for length in token_lengths]) / len(token_lengths)) ** 0.5
    return mean, std

# Generate synthetic data using the fine-tuned GPT model and seed texts

def generate_synthetic_data(tokenizer, model, seed_texts, n_samples, temperature, mean, std):
    generated_samples = []
    max_sequence_length = model.config.n_ctx

    # Set do_sample based on temperature
    do_sample = True if temperature != 1.0 else False

    for _ in range(n_samples):
        seed_text = random.choice(seed_texts)
        input_ids = tokenizer.encode(seed_text, return_tensors="pt").to(model.device)

        input_ids = input_ids[:, :max_sequence_length]

        target_length = int(np.random.normal(mean, std))+50
        if target_length < 1:
            target_length = 1

        output = model.generate(
            input_ids,
            max_length=target_length,
            num_return_sequences=1,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id,
            attention_mask=torch.ones(input_ids.shape, dtype=torch.long).to(model.device),
            do_sample=do_sample  # Set the do_sample parameter here
        )

        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

        # Remove the seed text
        seed_len = len(seed_text)
        gen_len = len(generated_text)
        generated_text = generated_text[seed_len:gen_len].strip()

        generated_samples.append(generated_text)

    return generated_samples


# Save the generated synthetic data to a new CSV file
def evaluate_and_save(df, generated_data, input_file, label_col, minority_class, text_col, model_name, temperature):
    synthetic_df = pd.DataFrame({label_col: [minority_class] * len(generated_data), text_col: generated_data})
    synthetic_df.to_csv(input_file.replace('.csv', f'_synthetic_data_{len(generated_data)}_{model_name}_temp{temperature}.csv'), index=False)

# Augment text using the Wordnet-based text augmenter
def augment_text(text, augmenter):
    return augmenter.augment(text)

# Prompt user to select the type of output dataset
def prompt_output_choice():
    while True:
        try:
            print("Select the type of output dataset:")
            print("1: Complete dataset with synthetic data")
            print("2: Only synthetic data")
            choice = int(input("Enter the number corresponding to your choice: ").strip())
            if choice in [1, 2]:
                return choice
            else:
                print("Invalid choice. Please enter 1 or 2.")
        except ValueError:
            print("Invalid input. Please enter an integer.")

# Main function to orchestrate the execution of the entire script
def main(output_choice=1):
    input_file, label_col, text_col = prompt_input()

    df = pd.read_csv(input_file)

    minority_classes, required_samples = balance_classes(df, label_col)

    model_name = select_model()

    print("\nEnter the temperature value for the generative algorithm (between 0 and 1).")
    print("A lower value (e.g., 0.1) will produce more focused and deterministic text, while a higher value (e.g., 0.8) will increase randomness and diversity.")
    print("Choosing a value too low might result in generated text that is too similar to the seed texts, and a value too high might produce less coherent text.")
    temperature = float(input("Temperature: "))

    augmentation_choice = input("Do you want to use data augmentation instead of complete message generation? (yes/no): ")

    synthetic_dfs = []

    for minority_class in minority_classes:
        seed_texts = df[df[label_col] == minority_class][text_col].sample(required_samples[minority_class], replace=True).tolist()

        with open("train_texts.txt", "w", encoding="utf-8") as f:
            f.write("\n".join(seed_texts))

        tokenizer, model = train_gpt2_model("train_texts.txt", model_name)

        # Calculate the mean and standard deviation of token lengths for the seed texts
        mean, std = get_mean_std(tokenizer, seed_texts)

        if augmentation_choice.lower() == "yes":
            augmenter = Wordnet()
            generated_data = [augment_text(text, augmenter) for text in seed_texts]
        else:
            generated_data = generate_synthetic_data(tokenizer, model, seed_texts, required_samples[minority_class], temperature, mean, std)

        synthetic_df = pd.DataFrame({label_col: [minority_class] * len(generated_data), text_col: generated_data})
        synthetic_dfs.append(synthetic_df)

    all_synthetic_df = pd.concat(synthetic_dfs, ignore_index=True)

    if output_choice == 1:
        balanced_df = pd.concat([df, all_synthetic_df], ignore_index=True)
        save_file_name = input_file.replace('.csv', f'_balanced_data_{len(balanced_df)}_{model_name}_temp{temperature}.csv')
        balanced_df.to_csv(save_file_name, index=False)
    else:
        save_file_name = input_file.replace('.csv', f'_synthetic_data_{len(all_synthetic_df)}_{model_name}_temp{temperature}.csv')
        all_synthetic_df.to_csv(save_file_name, index=False)

    if not os.path.exists(nltk.data.find('tokenizers/punkt')):
        nltk.download('punkt')
    bleu_scores = [sentence_bleu([seed_text.split()], generated_text.split()) for seed_text, generated_text in zip(seed_texts, generated_data)]
    print(f"Average BLEU score: {sum(bleu_scores) / len(bleu_scores)}")

if __name__ == "__main__":
    output_choice = prompt_output_choice()
    main(output_choice)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Select the type of output dataset:
1: Complete dataset with synthetic data
2: Only synthetic data
Enter the number corresponding to your choice: 1
Enter the path to the input file: /content/drive/MyDrive/Production Datasets/1_Praxis_Dataset/DeDuped_Dataset/Split Dataset/Training Data/Dataset_5971_deduped_training - Copy (2).csv
Enter the column name for the labels: LABEL
Enter the column name for the text: TEXT
Select the GPT model to use:
1: gpt2
2: gpt2-medium
3: gpt2-large
4: gpt2-xl
Enter the number corresponding to the model you want to use: 2

Enter the temperature value for the generative algorithm (between 0 and 1).
A lower value (e.g., 0.1) will produce more focused and deterministic text, while a higher value (e.g., 0.8) will increase randomness and diversity.
Choosing a value too low might result in generated text that is too similar to the seed texts, and a value too high might produce less coherent text.
Temperature: 1
Do you want to use data augmentation instead of comple

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2943 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/3042 [00:00<?, ? examples/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
100,4.114000
200,2.426500
300,1.264700
400,0.662200
500,0.376200
600,0.234800
700,0.183600
800,0.154600
900,0.116900
1000,0.100700


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Average BLEU score: 0.0003673670742395871


In [8]:
#>>>>>IMPORTANT<<<<<
#This code modified to attempt to shape the frequency distribution of the synthetic data
import torch
import pandas as pd
import random
import nltk
import os
import nltk
import numpy as np
import warnings
from transformers import logging
logging.set_verbosity_warning()
warnings.filterwarnings("ignore", category=UserWarning, message="Input length of input_ids is .* but `max_length` is set to .*")
from nltk.translate.bleu_score import sentence_bleu
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers.optimization import AdamW
from datasets import load_dataset
from textaugment import Wordnet
from collections import Counter
import numpy as np

nltk.download('punkt')

# Prompt user to input required information
def prompt_input():
    while True:
        try:
            input_file = input("Enter the path to the input file: ").strip()
            if not os.path.exists(input_file):
                print("The file does not exist. Please try again.")
                continue
            label_col = input("Enter the column name for the labels: ").strip()
            text_col = input("Enter the column name for the text: ").strip()
            return input_file, label_col, text_col
        except Exception as e:
            print(f"An error occurred: {e}. Please try again.")

# Calculate the imbalance of classes and determine the required samples for minority classes
def balance_classes(df, label_col):
    class_counts = df[label_col].value_counts()
    max_count = class_counts.max()

    minority_classes = class_counts[class_counts < max_count].index.tolist()
    required_samples = {cls: max_count - count for cls, count in zip(class_counts.index, class_counts) if cls in minority_classes}

    return minority_classes, required_samples

# Calculate word frequencies in a list of texts
def get_word_frequencies(texts):
    all_words = ' '.join(texts).split()
    word_frequencies = Counter(all_words)
    return word_frequencies

# Get the Z-score of a word based on its frequency
def get_z_score(word, word_frequencies, mean, std):
    return (word_frequencies[word] - mean) / std

# Let the user select the GPT model to use for synthetic text generation
def select_model():
    model_choices = {
        1: "gpt2",
        2: "gpt2-medium",
        3: "gpt2-large",
        4: "gpt2-xl",
    }

    print("Select the GPT model to use:")
    for key, value in model_choices.items():
        print(f"{key}: {value}")

    while True:
        try:
            selected = int(input("Enter the number corresponding to the model you want to use: "))
            return model_choices[selected]
        except (ValueError, KeyError):
            print("Invalid input. Please enter a valid model number.")

# Fine-tune the GPT model with the provided seed texts
def train_gpt2_model(train_texts, model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    train_dataset = load_dataset("text", data_files=train_texts)["train"]
    train_dataset = train_dataset.map(lambda x: tokenizer(x['text'], truncation=True, padding=True), batched=True)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    model = AutoModelForCausalLM.from_pretrained(model_name)

    training_args = TrainingArguments(
        output_dir="./gpt2_finetuned",
        overwrite_output_dir=True,
        num_train_epochs=3,
        per_device_train_batch_size=8,
        save_steps=10_000,
        save_total_limit=2,
        learning_rate=5e-5,
        logging_steps=100,
        logging_dir="./logs",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )

    trainer.train()

    return tokenizer, model

def get_mean_std(tokenizer, texts):
    token_lengths = [len(tokenizer.encode(text)) for text in texts]
    mean = sum(token_lengths) / len(token_lengths)
    std = (sum([(length - mean) ** 2 for length in token_lengths]) / len(token_lengths)) ** 0.5
    return mean, std

# Generate synthetic data using the fine-tuned GPT model and seed texts

def generate_synthetic_data(tokenizer, model, seed_texts, n_samples, temperature, mean, std):
    generated_samples = []
    max_sequence_length = model.config.n_ctx

    # Set do_sample based on temperature
    do_sample = True if temperature != 1.0 else False

    for _ in range(n_samples):
        seed_text = random.choice(seed_texts)
        input_ids = tokenizer.encode(seed_text, return_tensors="pt").to(model.device)

        input_ids = input_ids[:, :max_sequence_length]

        target_length = int(np.random.normal(mean, std))
        if target_length < 1:
            target_length = 1

        output = model.generate(
            input_ids,
            max_length=target_length,
            num_return_sequences=1,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id,
            attention_mask=torch.ones(input_ids.shape, dtype=torch.long).to(model.device),
            do_sample=do_sample  # Set the do_sample parameter here
        )

        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

        # Remove the seed text
        seed_len = len(seed_text)
        gen_len = len(generated_text)
        generated_text = generated_text[seed_len:gen_len].strip()

        generated_samples.append(generated_text)

    return generated_samples


# Save the generated synthetic data to a new CSV file
def evaluate_and_save(df, generated_data, input_file, label_col, minority_class, text_col, model_name, temperature):
    synthetic_df = pd.DataFrame({label_col: [minority_class] * len(generated_data), text_col: generated_data})
    synthetic_df.to_csv(input_file.replace('.csv', f'_synthetic_data_{len(generated_data)}_{model_name}_temp{temperature}.csv'), index=False)

# Augment text using the Wordnet-based text augmenter
def augment_text(text, augmenter):
    return augmenter.augment(text)

# Prompt user to select the type of output dataset
def prompt_output_choice():
    while True:
        try:
            print("Select the type of output dataset:")
            print("1: Complete dataset with synthetic data")
            print("2: Only synthetic data")
            choice = int(input("Enter the number corresponding to your choice: ").strip())
            if choice in [1, 2]:
                return choice
            else:
                print("Invalid choice. Please enter 1 or 2.")
        except ValueError:
            print("Invalid input. Please enter an integer.")

# Main function to orchestrate the execution of the entire script
def main(output_choice=1):
    input_file, label_col, text_col = prompt_input()
    df = pd.read_csv(input_file)

    # Calculate the word frequency distribution of the original data
    original_texts = df[text_col].tolist()
    original_word_frequencies = get_word_frequencies(original_texts)
    total_count = sum(original_word_frequencies.values())
    mean_freq = total_count / len(original_word_frequencies)
    std_freq = np.std(list(original_word_frequencies.values()))

    minority_classes, required_samples = balance_classes(df, label_col)
    model_name = select_model()

    temperature = float(input("Temperature: "))
    augmentation_choice = input("Do you want to use data augmentation? (yes/no): ")

    synthetic_dfs = []

    for minority_class in minority_classes:
        seed_texts = df[df[label_col] == minority_class][text_col].sample(required_samples[minority_class], replace=True).tolist()

        with open("train_texts.txt", "w", encoding="utf-8") as f:
            f.write("\n".join(seed_texts))

        tokenizer, model = train_gpt2_model("train_texts.txt", model_name)
        mean, std = get_mean_std(tokenizer, seed_texts)

        if augmentation_choice.lower() == "yes":
            # existing code for augmentation
            pass
        else:
            generated_data = generate_synthetic_data(tokenizer, model, seed_texts, required_samples[minority_class], temperature, mean, std)

        # Check if synthetic data maintains word frequencies
        synthetic_word_frequencies = get_word_frequencies(generated_data)

        for word in synthetic_word_frequencies.keys():
            z_score = get_z_score(word, synthetic_word_frequencies, mean_freq, std_freq)
            # Apply some logic based on z_score to increase or decrease frequency of the word in synthetic data
            # For example, you can choose to duplicate or remove the word based on its z_score

        synthetic_df = pd.DataFrame({label_col: [minority_class] * len(generated_data), text_col: generated_data})
        synthetic_dfs.append(synthetic_df)

    all_synthetic_df = pd.concat(synthetic_dfs, ignore_index=True)

    if output_choice == 1:
        balanced_df = pd.concat([df, all_synthetic_df], ignore_index=True)
        save_file_name = input_file.replace('.csv', f'_balanced_data_{len(balanced_df)}_{model_name}_temp{temperature}.csv')
        balanced_df.to_csv(save_file_name, index=False)
    else:
        save_file_name = input_file.replace('.csv', f'_synthetic_data_{len(all_synthetic_df)}_{model_name}_temp{temperature}.csv')
        all_synthetic_df.to_csv(save_file_name, index=False)

    if not os.path.exists(nltk.data.find('tokenizers/punkt')):
        nltk.download('punkt')

    bleu_scores = [sentence_bleu([seed_text.split()], generated_text.split()) for seed_text, generated_text in zip(seed_texts, generated_data)]
    print(f"Average BLEU score: {sum(bleu_scores) / len(bleu_scores)}")

if __name__ == "__main__":
    output_choice = prompt_output_choice()
    main(output_choice)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Select the type of output dataset:
1: Complete dataset with synthetic data
2: Only synthetic data
Enter the number corresponding to your choice: 1
Enter the path to the input file: /content/drive/MyDrive/Production Datasets/1_Praxis_Dataset/DeDuped_Dataset/Split Dataset/Training Data/Dataset_5971_deduped_training - Copy.csv
Enter the column name for the labels: LABEL
Enter the column name for the text: TEXT
Select the GPT model to use:
1: gpt2
2: gpt2-medium
3: gpt2-large
4: gpt2-xl
Enter the number corresponding to the model you want to use: 1
Temperature: .5
Do you want to use data augmentation? (yes/no): no


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2943 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/3042 [00:00<?, ? examples/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
100,4.812800
200,3.630800
300,2.700300
400,1.989000
500,1.528300
600,1.093200
700,0.912400
800,0.767800
900,0.588000
1000,0.549800


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Average BLEU score: 6.509472131356149e-05
